### Данный Jupiter-ноутбук предназначен для запуска на Jupyter-сервере внутри ML Space. Если вы хотите выполнить те же самые действия из удаленного Jupyter-сервера, посмотрите пример отправки задач на обучение через Training Job API.

# 0. Подключение библиотеки для работы с кластером и сервисом

In [1]:
try:
    import client_lib
except ImportError:
    raise RuntimeError("Скрипт не предназначен для запуска вне кластера")

In [2]:
import pathlib
# BASE_DIR will be like '/home/jovyan/DemoExample/'
BASE_DIR = pathlib.Path().absolute()
print(f"Working dir: {BASE_DIR}")

Working dir: /home/jovyan/quick-start/job_launch_cpu


In [3]:
import requests

In [4]:
def save_file(url, filename):
    # Download file and place it on local storage
    r = requests.get(url)
    
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f"{filename} downloaded from {url}")

# 1. Размещаем модель и данные на локальные диски кластера 

По умолчанию рабочая директория пользователя установлена в папку `/home/jovyan`

In [5]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_cpu/cal_housing_py3.pkz", "cal_housing_py3.pkz")

cal_housing_py3.pkz downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_cpu/cal_housing_py3.pkz


In [6]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_cpu/requirements.txt", "requirements.txt")

requirements.txt downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_cpu/requirements.txt


In [7]:
save_file("https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch_cpu/xgboost-boston-house-price.py", "xgboost-boston-house-price.py")

xgboost-boston-house-price.py downloaded from https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch_cpu/xgboost-boston-house-price.py


# 2. Сборка кастомного образа с нужными библиотеками

##### Содержимое файла requirements.txt

In [8]:
%cat /home/jovyan/quick-start/job_launch_cpu/requirements.txt

# tensorflow-gpu 1.15 already installed in registry.aicloud.sbcp.ru/horovod-tf15
# with other libraries, we add one more:
requests

##### Запуск сборки кастомного образа с необходимыми библиотеками

In [8]:
job = client_lib.ImageBuildJob(
                    from_image='cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0', # базовый образ
                    requirements_file=f'{BASE_DIR}/requirements.txt' # файл с зависимостями для кастомного образа
)

In [14]:
job.submit()

Files from nfs Christofari.V100 will be used in process. If you want to use files from other regions, you should use copy_to_nfs function. Do you want to continue? y/n:  y


'ImageBuildJob "{\'image\': \'cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-20843a\', \'name\': \'image-build-job-mvx22\', \'status\': \'ok\'}" created'

In [15]:
job.new_image # идентификатор кастомного образа

'cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-20843a'

In [ ]:
job.logs() # просмотр логов сборки образа в интерактивном режиме

# 3. Запуск задачи обучения на кластере

#### Создание задачи обучения и отправка ее на кластер

Параметр `base_image` предназначен для выбора Docker образа на котором будет запущена задача обчучения. Вы можете выбрать образ от ML Space или запустить задачу на своем кастомном образе (см. пункт 2 выше). Переменная `job.new_image` содержит название собранного кастомного образа (вида: `cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-20843a`).

По окончании сборки кастомного образа его можно указать в поле `base_image` и отправить задачу обучения на кластер с использованием этого образа:
```python
mnist_tf_run = client_lib.Job(base_image=job.new_image, # кастомный образ
                              script='/home/jovyan/quick-start/job_launch/tensorflow_mnist_estimator.py',
                             ...)
```

Обязательно указав парметры:
 * `type="cce"`
 * `region='CCE-MT'`
 * `instance_type` — конфигурация вычислительных ресурсов

**Важно.** Убедитесь, что код примеров которые вы запускаете, находится на файловой системе в нужном кластере. Например, в данном случае, файл `xgboost-boston-house-price.py` находится на файловой системе кластера `SR.01`.

In [5]:
# base_image='registry.aicloud.sbcp.ru/base/horovod-cuda10.0-tf1.15.0' -
#     базовый образ, можно заменить на свой (смотри выше)
cpu_job = client_lib.Job(
    base_image='cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda10.0-tf1.15.0-pt1.3.0',
    script=f'{BASE_DIR}/xgboost-boston-house-price.py',
    n_workers=1,
    type="cce",
    region='CCE-MT',
    instance_type='m3.large.8.linux'
    )

#### В предыдущей строке мы указали образ, в рамках которого будет исполняться задача (base_image), скрипт, который будет запущен (script), а также количество памяти и ядер процессора.

#### В этом случае под задача выполняется на 1 рабочем узле.

#### Теперь вызовем на объекте метод cpu_job.submit() и отправим задачу в очередь исполнения на кластере

In [6]:
cpu_job.submit()

'Job "lm-mpi-job-e94f9696-73d1-4324-9c63-ae860d965a86" created'

#### Вы можете просмотреть список ваших задач и их статус с помощью следующего метода

In [7]:
client_lib.jobs(region='CCE-MT')

2022-07-15T14:12:53Z : lm-mpi-job-151d3bcd-6a35-4fbe-913d-8bdf916864a0 : Failed
2022-07-15T11:17:09Z : lm-mpi-job-1ff575a9-3936-4089-95ab-73a3b3e9bffb : Failed
2022-07-15T13:53:02Z : lm-mpi-job-3789c8b4-54ee-4d66-b8c6-b4296d4bb64d : Failed
2022-07-15T13:22:25Z : lm-mpi-job-572826c3-6b41-431f-a414-9283544303d2 : Completed
2022-07-15T13:52:31Z : lm-mpi-job-73906e09-fa93-42cd-a9af-356d8caebd5b : Failed
2022-07-15T14:09:53Z : lm-mpi-job-758b154d-a935-4abd-a315-26073f17e995 : Failed
2022-07-15T13:27:34Z : lm-mpi-job-8304b7c2-0d40-4dca-9c4f-aacebdc3e7d4 : Failed
2022-07-11T18:41:33Z : lm-mpi-job-9df536fd-093d-487a-a54f-52fd8ae5a88a : Failed
2022-07-15T13:44:22Z : lm-mpi-job-c0d68be6-c927-4661-b122-d01938830a89 : Completed
2022-07-15T14:04:15Z : lm-mpi-job-dcabecab-5b83-436c-b1e8-75efda7a40c0 : Failed
2022-07-15T14:14:05Z : lm-mpi-job-e2b40c19-38ee-43e1-b599-2005ab4e4af3 : Completed
2022-07-15T14:50:36Z : lm-mpi-job-e94f9696-73d1-4324-9c63-ae860d965a86 : Pending


#### Логи задачи

In [8]:
cpu_job.logs()

2022-07-15T14:50:55.346197891Z Working dir: /home/jovyan
2022-07-15T14:50:55.346247597Z <class 'pandas.core.frame.DataFrame'>
2022-07-15T14:50:55.346252896Z RangeIndex: 20640 entries, 0 to 20639
2022-07-15T14:50:55.346255359Z Data columns (total 9 columns):
2022-07-15T14:50:55.346257579Z  #   Column      Non-Null Count  Dtype  
2022-07-15T14:50:55.346260905Z ---  ------      --------------  -----  
2022-07-15T14:50:55.346264042Z  0   MedInc      20640 non-null  float64
2022-07-15T14:50:55.346267538Z  1   HouseAge    20640 non-null  float64
2022-07-15T14:50:55.346270844Z  2   AveRooms    20640 non-null  float64
2022-07-15T14:50:55.346274663Z  3   AveBedrms   20640 non-null  float64
2022-07-15T14:50:55.346277755Z  4   Population  20640 non-null  float64
2022-07-15T14:50:55.346280993Z  5   AveOccup    20640 non-null  float64
2022-07-15T14:50:55.346284017Z  6   Latitude    20640 non-null  float64
2022-07-15T14:50:55.346287036Z  7   Longitude   20640 non-null  float64
2022-07-15T14:50:55.34

##### или по названию задачи

In [9]:
client_lib.logs("lm-mpi-job-e94f9696-73d1-4324-9c63-ae860d965a86", region='CCE-MT')

2022-07-15T14:50:55.346197891Z Working dir: /home/jovyan
2022-07-15T14:50:55.346247597Z <class 'pandas.core.frame.DataFrame'>
2022-07-15T14:50:55.346252896Z RangeIndex: 20640 entries, 0 to 20639
2022-07-15T14:50:55.346255359Z Data columns (total 9 columns):
2022-07-15T14:50:55.346257579Z  #   Column      Non-Null Count  Dtype  
2022-07-15T14:50:55.346260905Z ---  ------      --------------  -----  
2022-07-15T14:50:55.346264042Z  0   MedInc      20640 non-null  float64
2022-07-15T14:50:55.346267538Z  1   HouseAge    20640 non-null  float64
2022-07-15T14:50:55.346270844Z  2   AveRooms    20640 non-null  float64
2022-07-15T14:50:55.346274663Z  3   AveBedrms   20640 non-null  float64
2022-07-15T14:50:55.346277755Z  4   Population  20640 non-null  float64
2022-07-15T14:50:55.346280993Z  5   AveOccup    20640 non-null  float64
2022-07-15T14:50:55.346284017Z  6   Latitude    20640 non-null  float64
2022-07-15T14:50:55.346287036Z  7   Longitude   20640 non-null  float64
2022-07-15T14:50:55.34

#### (ЕСЛИ НЕОБХОДИМО ПРЕРВАТЬ ЗАДАЧУ) Варианты остановки задачи

In [10]:
cpu_job.kill()

'Job "lm-mpi-job-e94f9696-73d1-4324-9c63-ae860d965a86" deleted'

In [11]:
client_lib.kill('lm-mpi-job-e94f9696-73d1-4324-9c63-ae860d965a86')

'Job "lm-mpi-job-e94f9696-73d1-4324-9c63-ae860d965a86" deleted'

# 4. Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели пользователь сохраняет промежуточные результаты (checkpoints) обучения их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3. Внутри кода задачи обучения можно сохранять метрики модели с помощью mlflow (пример — в коде исполняемого файла `xgboost-boston-house-price.py`).